In [1]:
!hostname

chinook04.rcs.alaska.edu


In [2]:
import xarray as xr
import os
import glob
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import regionmask
import matplotlib.pyplot as plt

FIRST FOR 31 km ERA5
-

In [3]:
#LOADING IN ALL THE DATA 
shapefile_path = "/center1/DYNDOWN/phutton5/ROS/boundaries/Alaska_Borough_and_Census_Area_Boundaries.shp"
borough_boundaries = gpd.read_file(shapefile_path)
borough_boundaries = borough_boundaries.set_crs(epsg=3338)
borough_boundaries = borough_boundaries.to_crs(epsg=4326)
FNSB_boundary = borough_boundaries[borough_boundaries['CommunityN'] == 'Fairbanks North Star Borough']
FNSB_geom = FNSB_boundary.geometry.iloc[0] 
FNSB_coords = []
FNSB_coords.extend(list(FNSB_geom.exterior.coords))
FNSB_coords = np.array(FNSB_coords)  
FNSB_coords = pd.DataFrame({
    "lon": FNSB_coords[:, 0],
    "lat": FNSB_coords[:, 1]
})

Fairbanks_lat=(64.84)
Fairbanks_lon=(-147.72)

Mun_of_Anc_boundary = borough_boundaries[borough_boundaries['CommunityN'] == 'Municipality of Anchorage']
Bethel_boundary = borough_boundaries[borough_boundaries['CommunityN'] == 'Bethel Census Area']
Mat_Su_boundary=borough_boundaries[borough_boundaries['CommunityN'] == 'Matanuska-Susitna Borough']

Denali_boundary=borough_boundaries[borough_boundaries['CommunityN'] == 'Denali Borough']


path='/center1/DYNDOWN/phutton5/ROS/Raw_ERA5/Raw_ERA5_netcdf'
pattern=f'Raw*.nc'
file_list = sorted(glob.glob(os.path.join(path, pattern)))
era5_31km = xr.open_mfdataset(file_list, combine='by_coords',engine='netcdf4')
#data = data.fillna(0)

seasons=era5_31km['season']
era531kmlat=era5_31km['latitude']
era531kmlon=era5_31km['longitude']

Defining Functions
-

In [4]:
def borough_land_mask(shapefile, data, lat, lon, borough_name):
    mask = regionmask.mask_geopandas(shapefile, lon, lat)
    data_masked = data.where(~np.isnan(mask))
    data_final=data_masked
    return data_final


In [5]:
def ros_normalized_per_borough(ds):
    ros = ds
    total_ros_hours = []
    n_pixels = []
    normalized = []

    for i in range(len(ros.season)):
        ros_season = ros.isel(season=i).sum(dim="month", skipna=True)

        total = ros_season.sum(skipna=True).compute()
        total_ros_hours.append(total)

        pixel_count = (ros_season.fillna(0) > 0).sum().compute()
        n_pixels.append(pixel_count)

        if pixel_count > 0 and not np.isnan(total):
            normalized.append(total / pixel_count)
        else:
            normalized.append(0)   # or np.nan if you prefer

    return {
        "total_ros_hours": total_ros_hours,
        "n_pixels": n_pixels,
        "normalized": normalized
    }


Running Functions 
-

In [6]:
#era5_31km_data_masked = borough_land_mask(borough_boundary, era5_31km, lat, lon)
era5_31km_masked = {}   
for borough_name in borough_boundaries['CommunityN'].unique():
    #print(f"Processing {borough_name}...")
    shapefile = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
    masked = borough_land_mask(shapefile, era5_31km, era531kmlat, era531kmlon, borough_name)
    ros_only = masked["ros_tally"]
    era5_31km_masked[borough_name] = ros_only


In [7]:
borough_ros_norm = {}
era5_31km_masked = xr.Dataset(era5_31km_masked)

for borough_name, da in era5_31km_masked.items():
    borough_ros_norm[borough_name] = ros_normalized_per_borough(da)
    
borough_list = list(borough_ros_norm.keys())
n_seasons = len(borough_ros_norm[borough_list[0]]['total_ros_hours'])

total_ros_array = np.array([borough_ros_norm[b]['total_ros_hours'] for b in borough_list])
n_pixels_array   = np.array([borough_ros_norm[b]['n_pixels'] for b in borough_list])
normalized_array = np.array([borough_ros_norm[b]['normalized'] for b in borough_list])

ds_out = xr.Dataset(
    data_vars=dict(
        total_ros_hours=(["borough", "season"], total_ros_array),
        n_pixels=(["borough", "season"], n_pixels_array),
        normalized=(["borough", "season"], normalized_array),
    ),
    coords=dict(
        borough=borough_list,
        season=np.arange(n_seasons)
    )
)

ds_out.to_netcdf("/center1/DYNDOWN/phutton5/ROS/Raw_ERA5/Raw_ERA5_netcdf/Raw_ERA5_normalzed_per_borough.nc")
print("Saved ROS stats to NetCDF!")

Saving Data

NOW REPEAT FOR 4km ERA5
-

In [14]:
#path='/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/'
#pattern = 'ROS_*.nc'
#file_list = sorted(glob.glob(os.path.join(path, pattern)))
#all_AK_era5 = xr.open_mfdataset(file_list, combine='by_coords', engine='netcdf4')

all_AK_era5=xr.open_dataset('/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/NEAREST_REGRIDDED_ERA5_4km_ROS.nc')
#BILINEAR_REGRIDDED_ERA5_4km_ROS.nc')

shapefile_path = "/center1/DYNDOWN/phutton5/ROS/boundaries/Alaska_Borough_and_Census_Area_Boundaries.shp"
borough_boundaries = gpd.read_file(shapefile_path)
borough_boundaries = borough_boundaries.set_crs(epsg=3338).to_crs(epsg=4326)

borough_boundary = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
print(f"Processing for {borough_name}")

geom = borough_boundary.geometry.iloc[0]
#coords = np.array(list(geom.exterior.coords))
#coords_df = pd.DataFrame({"lon": coords[:, 0], "lat": coords[:, 1]})

lat = all_AK_era5['latitude']
lon = all_AK_era5['longitude']

geo_em_path = "/import/beegfs/CMIP6/wrf_era5/geo_em.d02.nc"
geo = xr.open_dataset(geo_em_path)
landmask = geo['LANDMASK'].squeeze(dim="Time")
#landmask_expanded = landmask.expand_dims(Time=data_masked.Time)

Processing for Yukon-Koyukuk Census Area


In [24]:
def borough_land_mask(shapefile, data, lat, lon,borough_name):
    mask = regionmask.mask_geopandas(shapefile, lon, lat)
    data_masked = data.where(~np.isnan(mask))
    geo_em_path = "/import/beegfs/CMIP6/wrf_era5/geo_em.d02.nc"
    geo = xr.open_dataset(geo_em_path)
    landmask = geo['LANDMASK'].squeeze(dim="Time")
    #   landmask_expanded = landmask.expand_dims(Time=data_masked.Time)

    # Apply land mask and fill missing data
    data_fixed = data_masked.where(landmask == 1)
    data_fixed = data_fixed.chunk(dict(south_north=-1, west_east=-1))
    data_filled = data_fixed.interpolate_na(dim="south_north", method="nearest", fill_value="extrapolate")
    data_filled = data_filled.interpolate_na(dim="west_east", method="nearest", fill_value="extrapolate")
    data_final = data_filled.where(~np.isnan(mask))
    print(f"Land mask complete for {borough_name}")
    return data_final

In [15]:
def normalizing(ds):
    total_ros_hours = []
    n_pixels = []
    normalized = []
    for i in range(len(ds.season)):
        ros_season = ds['ros_tally'].isel(season=i)
        ros_season = ros_season.sum(dim="month")         
        total = ros_season.sum().compute().item()
        total_ros_hours.append(total)
    
        pixels = (ros_season > 0).sum().compute().item()
        n_pixels.append(pixels)
        normalized.append(total / pixels if pixels > 0 else np.nan)

    return {
        "total_ros_hours": total_ros_hours,
        "n_pixels": n_pixels,
        "normalized": normalized
    }

Running the functions..
-

In [16]:
borough_names = borough_boundaries['CommunityN'].unique()
borough_ros_norm = {}
for borough_name in borough_names:
    borough_shape = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
    #data_masked = borough_land_mask(borough_shape, all_AK_era5, lat, lon, borough_name)
    ros_stats = normalizing(all_AK_era5)
    borough_ros_norm[borough_name] = ros_stats

Saving it!


In [13]:
borough_list = list(borough_ros_norm.keys())
n_seasons = len(next(iter(borough_ros_norm.values()))['total_ros_hours'])
total_ros_array = np.zeros((len(borough_list), n_seasons))
n_pixels_array = np.zeros((len(borough_list), n_seasons))
normalized_array = np.zeros((len(borough_list), n_seasons))
for i, borough_name in enumerate(borough_list):
    total_ros_array[i, :] = borough_ros_norm[borough_name]['total_ros_hours']
    n_pixels_array[i, :] = borough_ros_norm[borough_name]['n_pixels']
    normalized_array[i, :] = borough_ros_norm[borough_name]['normalized']

ds_ros = xr.Dataset(
    {
        "total_ros_hours": (["borough", "season"], total_ros_array),
        "n_pixels": (["borough", "season"], n_pixels_array),
        "normalized": (["borough", "season"], normalized_array)
    },
    coords={
        "borough": borough_list,
        "season": np.arange(n_seasons) 
    }
)
ds_ros.to_netcdf("/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/REGRIDDED_NEAREST_ros_stats_per_borough.nc")
print("Saved ROS stats to NetCDF!")


Saved ROS stats to NetCDF!


Trying to make i do this the same way twice...
-

In [19]:
import xarray as xr
import os
import glob
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import regionmask
import matplotlib.pyplot as plt

In [20]:
#Load in data 
nearest_regridded4km_era5=xr.open_dataset('/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/NEAREST_REGRIDDED_ERA5_4km_ROS.nc')
#BILINEAR_REGRIDDED_ERA5_4km_ROS.nc')

shapefile_path = "/center1/DYNDOWN/phutton5/ROS/boundaries/Alaska_Borough_and_Census_Area_Boundaries.shp"
borough_boundaries = gpd.read_file(shapefile_path)
borough_boundaries = borough_boundaries.set_crs(epsg=3338).to_crs(epsg=4326)

borough_boundary = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
print(f"Processing for {borough_name}")
#Fairbanks_lat=(64.84)
#Fairbanks_lon=(-147.72)

lat = nearest_regridded4km_era5['latitude']
lon = nearest_regridded4km_era5['longitude']

#LOADING IN ALL THE DATA for seasons list...
path='/center1/DYNDOWN/phutton5/ROS/Raw_ERA5/Raw_ERA5_netcdf'
pattern=f'Raw*.nc'
file_list = sorted(glob.glob(os.path.join(path, pattern)))
era5_31km = xr.open_mfdataset(file_list, combine='by_coords',engine='netcdf4')
seasons=era5_31km['season']

Processing for Yukon-Koyukuk Census Area


In [21]:
def borough_land_mask(shapefile, data, lat, lon, borough_name):
    mask = regionmask.mask_geopandas(shapefile, lon, lat)
    data_masked = data.where(~np.isnan(mask))
    data_final=data_masked
    return data_final


In [22]:
def ros_normalized_per_borough(ds):
    ros = ds
    total_ros_hours = []
    n_pixels = []
    normalized = []

    for i in range(len(ros.season)):
        ros_season = ros.isel(season=i).sum(dim="month", skipna=True)

        total = ros_season.sum(skipna=True).compute()
        total_ros_hours.append(total)

        pixel_count = (ros_season.fillna(0) > 0).sum().compute()
        n_pixels.append(pixel_count)

        if pixel_count > 0 and not np.isnan(total):
            normalized.append(total / pixel_count)
        else:
            normalized.append(0)   # or np.nan if you prefer

    return {
        "total_ros_hours": total_ros_hours,
        "n_pixels": n_pixels,
        "normalized": normalized
    }

In [23]:
era5_31km_masked = {}   
for borough_name in borough_boundaries['CommunityN'].unique():
    shapefile = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
    masked = borough_land_mask(shapefile, era5_31km, era531kmlat, era531kmlon, borough_name)
    ros_only = masked["ros_tally"]
    era5_31km_masked[borough_name] = ros_only

borough_ros_norm = {}
era5_31km_masked = xr.Dataset(era5_31km_masked)

for borough_name, da in era5_31km_masked.items():
    borough_ros_norm[borough_name] = ros_normalized_per_borough(da)
    
borough_list = list(borough_ros_norm.keys())
n_seasons = len(borough_ros_norm[borough_list[0]]['total_ros_hours'])

total_ros_array = np.array([borough_ros_norm[b]['total_ros_hours'] for b in borough_list])
n_pixels_array   = np.array([borough_ros_norm[b]['n_pixels'] for b in borough_list])
normalized_array = np.array([borough_ros_norm[b]['normalized'] for b in borough_list])

ds_out = xr.Dataset(
    data_vars=dict(
        total_ros_hours=(["borough", "season"], total_ros_array),
        n_pixels=(["borough", "season"], n_pixels_array),
        normalized=(["borough", "season"], normalized_array),
    ),
    coords=dict(
        borough=borough_list,
        season=np.arange(n_seasons)
    )
)

ds_out.to_netcdf("/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/TEST_Raw_ERA5_normalzed_per_borough.nc")
print("Saved ROS stats to NetCDF!")

Saved ROS stats to NetCDF!


In [25]:
nearest_regridded4km_era5_masked = {}   
for borough_name in borough_boundaries['CommunityN'].unique():
    shapefile = borough_boundaries[borough_boundaries['CommunityN'] == borough_name]
    masked = borough_land_mask(shapefile, nearest_regridded4km_era5, era531kmlat, era531kmlon, borough_name)
    ros_only = masked["ros_tally"]
    nearest_regridded4km_era5_masked[borough_name] = ros_only

borough_ros_norm = {}
nearest_regridded4km_era5_masked = xr.Dataset(nearest_regridded4km_era5_masked)

for borough_name, da in nearest_regridded4km_era5_masked.items():
    borough_ros_norm[borough_name] = ros_normalized_per_borough(da)
    
borough_list = list(borough_ros_norm.keys())
n_seasons = len(borough_ros_norm[borough_list[0]]['total_ros_hours'])

total_ros_array = np.array([borough_ros_norm[b]['total_ros_hours'] for b in borough_list])
n_pixels_array   = np.array([borough_ros_norm[b]['n_pixels'] for b in borough_list])
normalized_array = np.array([borough_ros_norm[b]['normalized'] for b in borough_list])

ds_out = xr.Dataset(
    data_vars=dict(
        total_ros_hours=(["borough", "season"], total_ros_array),
        n_pixels=(["borough", "season"], n_pixels_array),
        normalized=(["borough", "season"], normalized_array),
    ),
    coords=dict(
        borough=borough_list,
        season=np.arange(n_seasons)
    )
)

ds_out.to_netcdf("/center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/TEST_REGRID_NEAREST_ERA5_normalzed_per_borough.nc")
print("Saved ROS stats to NetCDF!")

Saved ROS stats to NetCDF!


In [26]:
#OLD
ros_hourly_totals_climatology = []
for i in range(len(data['ros_tally'].season)):
    val = data['ros_tally'].isel(season=i).sum().compute().item()
    ros_hourly_totals_climatology.append(val)

n_pixels_per_season = []

for i in range(len(data['ros_tally'].season)):
    # Get 2D field for the season
    ros_this_season = data['ros_tally'].isel(season=i)
    
    # Create a boolean mask of where ROS occurred (value > 0)
    grid_with_ros = ros_this_season > 0
    
    # Count how many grid cells had ROS
    n_pixels = grid_with_ros.sum().compute().item()
    n_pixels_per_season.append(n_pixels)

ros_hours_normalized = [total / n if n > 0 else np.nan 
                        for total, n in zip(ros_hourly_totals_climatology, n_pixels_per_season)]
seasons = [str(s) for s in data['ros_tally'].season.values]

In [27]:
import xarray as xr
import geopandas as gpd
import regionmask
import numpy as np
import glob
import os

# ============================================================
# LOAD BOROUGH SHAPEFILE
# ============================================================

shapefile_path = (
    "/center1/DYNDOWN/phutton5/ROS/boundaries/"
    "Alaska_Borough_and_Census_Area_Boundaries.shp"
)

borough_boundaries = gpd.read_file(shapefile_path)
borough_boundaries = borough_boundaries.set_crs(epsg=3338).to_crs(epsg=4326)
borough_names = borough_boundaries["CommunityN"].unique()

# ============================================================
# LOAD ERA5 31 KM DATA
# ============================================================

path_31km = "/center1/DYNDOWN/phutton5/ROS/Raw_ERA5/Raw_ERA5_netcdf"
files_31km = sorted(glob.glob(os.path.join(path_31km, "Raw*.nc")))

era5_31km = xr.open_mfdataset(
    files_31km,
    combine="by_coords",
    engine="netcdf4"
)

ros_31km = era5_31km["ros_tally"]
lat_31km = era5_31km["latitude"]
lon_31km = era5_31km["longitude"]

# ============================================================
# LOAD NEAREST-REGRIDDED ERA5 4 KM DATA
# ============================================================

era5_4km = xr.open_dataset(
    "/center1/DYNDOWN/phutton5/ROS/All_of_AK/"
    "All_of_AK_netcdf_files/NEAREST_REGRIDDED_ERA5_4km_ROS.nc"
)

ros_4km = era5_4km["ros_tally"]
lat_4km = era5_4km["latitude"]
lon_4km = era5_4km["longitude"]

# ============================================================
# HELPER FUNCTIONS
# ============================================================

def borough_mask(da, shapefile, lat, lon):
    mask = regionmask.mask_geopandas(shapefile, lon, lat)
    return da.where(~np.isnan(mask))

def ros_normalized_per_borough(ros_da):
    total_ros_hours = []
    n_pixels = []

    for i in range(ros_da.sizes["season"]):
        ros_season = ros_da.isel(season=i)

        total = ros_season.sum(skipna=True).compute().item()
        total_ros_hours.append(total)

        pixels = (ros_season > 0).sum(skipna=True).compute().item()
        n_pixels.append(pixels)

    normalized = [
        t / p if p > 0 else np.nan
        for t, p in zip(total_ros_hours, n_pixels)
    ]

    return total_ros_hours, n_pixels, normalized

# ============================================================
# CONFIG FOR BOTH DATASETS
# ============================================================

datasets = {
    "ERA5_31km": {
        "ros": ros_31km,
        "lat": lat_31km,
        "lon": lon_31km,
        "out": (
            "/center1/DYNDOWN/phutton5/ROS/All_of_AK/"
            "All_of_AK_netcdf_files/TEST2_ERA5_31km_normalized_per_borough.nc"
        )
    },
    "ERA5_4km": {
        "ros": ros_4km,
        "lat": lat_4km,
        "lon": lon_4km,
        "out": (
            "/center1/DYNDOWN/phutton5/ROS/All_of_AK/"
            "All_of_AK_netcdf_files/TEST2_ERA5_4km_normalized_per_borough.nc"
        )
    }
}

# ============================================================
# MAIN PROCESSING LOOP
# ============================================================

for label, cfg in datasets.items():

    print(f"\nProcessing {label}")

    total_ros_all = []
    n_pixels_all = []
    normalized_all = []

    for borough_name in borough_names:
        print(f"  {borough_name}")

        shape = borough_boundaries[
            borough_boundaries["CommunityN"] == borough_name
        ]

        ros_masked = borough_mask(
            cfg["ros"],
            shape,
            cfg["lat"],
            cfg["lon"]
        )

        total, pixels, norm = ros_normalized_per_borough(ros_masked)

        total_ros_all.append(total)
        n_pixels_all.append(pixels)
        normalized_all.append(norm)

    total_ros_all = np.array(total_ros_all)
    n_pixels_all = np.array(n_pixels_all)
    normalized_all = np.array(normalized_all)

    ds_out = xr.Dataset(
        data_vars=dict(
            total_ros_hours=(["borough", "season"], total_ros_all),
            n_pixels=(["borough", "season"], n_pixels_all),
            normalized=(["borough", "season"], normalized_all),
        ),
        coords=dict(
            borough=borough_names,
            season=cfg["ros"]["season"].values
        )
    )

    ds_out.to_netcdf(cfg["out"])
    print(f"Saved {label} → {cfg['out']}")

print("All processing complete.")



Processing ERA5_31km
  Aleutians East Borough
  Aleutians West Census Area
  Bethel Census Area
  Bristol Bay Borough
  Chugach Census Area
  City and Borough of Juneau
  City and Borough of Sitka
  City and Borough of Wrangell
  City and Borough of Yakutat
  Copper River Census Area
  Denali Borough
  Dillingham Census Area
  Fairbanks North Star Borough
  Haines Borough
  Hoonah-Angoon Census Area
  Kenai Peninsula Borough
  Ketchikan Gateway Borough
  Kodiak Island Borough
  Kusilvak Census Area
  Lake and Peninsula Borough
  Matanuska-Susitna Borough
  Municipality of Anchorage
  Municipality of Skagway
  Nome Census Area
  North Slope Borough
  Northwest Arctic Borough
  Petersburg Borough
  Prince of Wales-Hyder Census Area
  Southeast Fairbanks Census Area
  Yukon-Koyukuk Census Area
Saved ERA5_31km → /center1/DYNDOWN/phutton5/ROS/All_of_AK/All_of_AK_netcdf_files/TEST2_ERA5_31km_normalized_per_borough.nc

Processing ERA5_4km
  Aleutians East Borough
  Aleutians West Census Area